In [90]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
import sys, os, time
import h5py
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tensorflow_addons.optimizers import CyclicalLearningRate
import pickle

from utils import get_diag
from model import get_model

# from competition baseline
from base_code.helper import to_observed_matrix
from base_code.posterior_utils import compute_posterior_loss, default_prior_bounds
from base_code.submit_format import to_competition_format
from base_code.FM_utils_final import ariel_resolution, setup_dedicated_fm, initialise_forward_model
from base_code.spectral_metric import compute_spectral_loss

import taurex.log
taurex.log.disableLogging()

In [2]:
def get_data(ad, s):
    ad = ad.copy()
    spectra_rw_mean = s.mean(axis=1)
    spectra_rw_std = s.std(axis=1)
    spectra_rw_min = s.min(axis=1)
    spectra_rw_max = s.max(axis=1)
        
    ad['spectra_rw_mean'] = spectra_rw_mean
    ad['spectra_rw_std'] = spectra_rw_std
    ad['spectra_rw_min'] = spectra_rw_min
    ad['spectra_rw_max'] = spectra_rw_max

    ad['planet_mass_kg/star_mass_kg'] = ad['planet_mass_kg']/ad['star_mass_kg']
    ad['star_density'] = ad['star_mass_kg']*(4/3)*np.pi*(ad['star_radius_m']**3)
    ad['planet_eqlbm_temp'] = np.sqrt(ad['star_radius_m']/(2*ad['planet_distance']))*ad['star_temperature']
    ad['planet_semimajor_axis'] = ((ad['star_mass_kg'] + ad['planet_mass_kg'])*((ad['planet_orbital_period']/(2*np.pi))**2))**(1/3)

    cols = [c for c in ad.columns if c != 'planet_ID']
    for c in cols:
        ad[c] = np.log(ad[c])
        
    return ad

# Load training data and calculate metrics

In [3]:
# assume this order?
target_labels = ['planet_radius','planet_temp','log_H2O','log_CO2','log_CO','log_CH4','log_NH3']
num_targets = len(target_labels)

target_scales_dict = {
    'planet_radius': [0.1, 3],
    'planet_temp': [0, 7000],
    'log_H2O': [-12, -1],
    'log_CO2': [-12, -1],
    'log_CO': [-12, -1],
    'log_CH4': [-12, -1],
    'log_NH3': [-12, -1]
}

target_scales_arr = np.array([target_scales_dict[l] for l in target_labels]) # safer, in case dict unordered
min_vals = target_scales_arr[:, 0]
max_vals = target_scales_arr[:, 1]

In [ ]:
# adjust as needed
training_path = 'data/FullDataset/TrainingData'

In [4]:
training_GT_path = os.path.join(training_path, 'Ground Truth Package')
GT_trace_path = os.path.join(training_GT_path, 'Tracedata.hdf5')
trace_GT = h5py.File(os.path.join(training_GT_path, 'TraceData.hdf5'),"r")

spectral_training_data = h5py.File(os.path.join(training_path,'SpectralData.hdf5'),"r")
aux_data = pd.read_csv(os.path.join(training_path,'AuxillaryTable.csv'))
soft_label_data = pd.read_csv(os.path.join(training_GT_path, 'FM_Parameter_Table.csv'))

trace_file = h5py.File(GT_trace_path)

if 'Unnamed: 0' in soft_label_data.columns:
    soft_label_data = soft_label_data.drop(['Unnamed: 0'], axis=1)

In [5]:
spec_matrix_file = 'spec_matrix.npy'
if spec_matrix_file in os.listdir('data/'):
    print('opening existing spec matrix file')
    with open('data/spec_matrix.npy', 'rb') as f:
        spec_matrix = np.load(f)
    print('spectra shape:', spec_matrix.shape)
else:
    print('constructing spec matrix and saving to file')
    start_time = time.time()
    spec_matrix = to_observed_matrix(spectral_training_data, aux_data)
    with open('data/spec_matrix.npy', 'wb') as f:
        np.save(f, spec_matrix)
    print("finished after: {}, spectral matrix shape: {}".format(time.time() - start_time, spec_matrix.shape))

noise = spec_matrix[:, :, 2]
spectra = spec_matrix[:, :, 1]
wl_grid = spec_matrix[:, :, 0]
bin_width = spec_matrix[:, :, 3]

global_spectra_mean = np.mean(spectra)
global_spectra_std = np.std(spectra)
print(f'spectra mean: {global_spectra_mean}, std: {global_spectra_std}')

wl_channels = spectra.shape[1] # wavelength_channels

opening existing spec matrix file
spectra shape: (41423, 52, 4)
spectra mean: 0.008257788618509107, std: 0.011651996038057373


In [6]:
planet_desc_filename = 'planets_data_desc.csv'

if planet_desc_filename in os.listdir():
    print('reading from file')
    df_planet_has_data = pd.read_csv('planets_data_desc.csv')

else:
    print('building planet_data_desc')
    planet_list = [p for p in trace_file.keys()] 

    planet_data_existence = []
    for idx, pl in enumerate(planet_list):
        # print(trace_file[pl]['weights'].shape != ())
        has_data = trace_file[pl]['weights'].shape != ()
        planet_data_existence.append((pl, has_data))
        
    print(f'finished, total planets: {len(planet_data_existence)}')
    
    # convenience stuff
    df_planet_has_data = pd.DataFrame(planet_data_existence, columns=['planet_id_str', 'has_trace_data'])
    df_planet_has_data['planet_id'] = df_planet_has_data['planet_id_str'].str \
                                                                         .replace('Planet_train', '') \
                                                                         .astype(int)
    df_planet_has_data['planet_ID'] = df_planet_has_data['planet_id_str'].str.replace('Planet_', '')
    df_planet_has_data = df_planet_has_data.sort_values(by=['planet_id']).reset_index(drop=True)
    
    df_planet_has_data.to_csv(planet_desc_filename, index=False)
    
total_with_data = df_planet_has_data['has_trace_data'].sum()
print(f'has_data: {total_with_data}, no_data: {df_planet_has_data.shape[0] - total_with_data}')
df_planet_has_data.head()

reading from file
has_data: 6766, no_data: 34657


,planet_id_str,has_trace_data,planet_id,planet_ID
0,Planet_train1,True,1,train1
1,Planet_train2,True,2,train2
2,Planet_train3,True,3,train3
3,Planet_train4,False,4,train4
4,Planet_train5,True,5,train5


In [7]:
cols_for_split = ['planet_ID', 'planet_temp', 'planet_radius']
soft_d_has_trace = soft_label_data.merge(df_planet_has_data, on='planet_ID')
soft_d_has_trace.head()

,planet_ID,planet_radius,planet_temp,log_H2O,log_CO2,log_CO,log_CH4,log_NH3,planet_id_str,has_trace_data,planet_id
0,train1,0.559620,863.394770,-8.865868,-6.700707,-5.557561,-8.957615,-3.097540,Planet_train1,True,1
1,train2,1.118308,1201.700465,-4.510258,-8.228966,-3.565427,-7.807424,-3.633658,Planet_train2,True,2
2,train3,0.400881,1556.096477,-7.225472,-6.931472,-3.081975,-8.567854,-5.378472,Planet_train3,True,3
3,train4,0.345974,1268.624884,-7.461157,-5.853334,-3.044711,-5.149378,-3.815568,Planet_train4,False,4
4,train5,0.733184,1707.323564,-4.140844,-7.460278,-3.181793,-5.996593,-4.535345,Planet_train5,True,5


In [8]:
train, test = train_test_split(soft_d_has_trace, 
                train_size=0.9,
                random_state=42)
print(f'train_shape: {train.shape}, test_shape: {test.shape}')
train.head()

train_shape: (37280, 11), test_shape: (4143, 11)


,planet_ID,planet_radius,planet_temp,log_H2O,log_CO2,log_CO,log_CH4,log_NH3,planet_id_str,has_trace_data,planet_id
24311,train24312,0.173529,807.875012,-4.835660,-8.588479,-4.651833,-5.617772,-7.336094,Planet_train24312,False,24312
1107,train1108,0.156580,611.859047,-6.332634,-4.279123,-3.590717,-4.503540,-4.024263,Planet_train1108,False,1108
39362,train39363,0.959320,1615.958900,-8.835347,-5.211572,-5.303128,-7.085874,-4.611678,Planet_train39363,True,39363
1133,train1134,0.223296,871.099229,-5.845370,-7.554668,-4.676774,-4.889590,-6.492952,Planet_train1134,False,1134
7932,train7933,0.146215,933.031771,-3.212717,-6.946803,-3.745800,-7.427348,-6.925160,Planet_train7933,False,7933


In [31]:
train_idx = train.loc[train['has_trace_data']].index # train.index
test_idx = test.loc[test['has_trace_data']].index # test.index

test_spectra = spectra[test_idx]

# Guassian Multivariate check

In [59]:
scaler_pickles = [
    'multi_gauss_scaler_fold_0_of_5.pickle',
    'multi_gauss_scaler_fold_1_of_5.pickle',
    'multi_gauss_scaler_fold_2_of_5.pickle',
    'multi_gauss_scaler_fold_3_of_5.pickle',
    'multi_gauss_scaler_fold_4_of_5.pickle'
]

model_checkpoints = [
    './weights/stage_2_gauss_multivariate_fold_0_of_5',
    './weights/stage_2_gauss_multivariate_fold_1_of_5',
    './weights/stage_2_gauss_multivariate_fold_2_of_5',
    './weights/stage_2_gauss_multivariate_fold_3_of_5',
    './weights/stage_2_gauss_multivariate_fold_4_of_5'
]

In [33]:
ax = get_data(aux_data, spectra, method='kfold')
len(ax.columns)

17

In [60]:
loss = 'chol'
n_feature_cols = None
num_spectra_features = wl_channels
num_added_features = 16 # len(aux_columns) - 1

model = get_model(loss, num_spectra_features, n_feature_cols, num_added_features, num_targets)

instances = 4500 # 2500 + 500 + 1000 + 500
g_y_pred_test = np.zeros((len(model_checkpoints)*instances, len(test_idx), num_targets))
    
for j, (scaler_file, model_ckpt) in enumerate(zip(scaler_pickles, model_checkpoints)):
    start_time = time.time()
    with open('saved_objects/' + scaler_file, 'rb') as f:
        aux_data_scaler = pickle.load(f)
        
    _aux_data = get_data(aux_data, spectra)
    aux_columns = _aux_data.columns
    test_aux_data = _aux_data[aux_columns].iloc[test_idx]
    
    # num_added_features = len(aux_columns) - 1
    std_test_aux_data = aux_data_scaler.transform(test_aux_data.drop(['planet_ID'], axis=1).values)
    
    print(f'loading {model_ckpt}')
    model.load_weights(model_ckpt).expect_partial()
    
    _test_spectra = test_spectra - np.mean(test_spectra, axis=1).reshape((-1, 1))
    _x_data = [_test_spectra, std_test_aux_data]
    gauss_y_pred = model.predict(_x_data, verbose=1, batch_size=5000) # will this batch size work??
    
    mu = gauss_y_pred[:, :num_targets]
    L  = gauss_y_pred[:, num_targets:]
    L, diag = get_diag(gauss_y_pred, num_targets, gauss_y_pred.shape[0])
    LT = tf.transpose(L, perm=[0,2,1])
    Sigma = tf.linalg.inv(tf.matmul(L, LT))

    for i in range(len(test_idx)):
        _y_pred_sample = np.random.multivariate_normal(mu[i], Sigma[i], instances)
        g_y_pred_test[j*instances:(j+1)*instances, i, :] += _y_pred_sample
        
    print(f'finished {j} after {time.time() - start_time}, rows: {len(test_idx)}, samples: {instances}')
        
del(model)
tf.keras.backend.clear_session()

loading ./weights/stage_2_gauss_multivariate_fold_0_of_5
1/1 [==============================] - 0s 478ms/step


C:\Users\andre\AppData\Local\Temp\ipykernel_30688\102697444.py:42: RuntimeWarning: covariance is not positive-semidefinite.
  _y_pred_sample = np.random.multivariate_normal(mu[i], Sigma[i], instances)


finished 0 after 1.8783197402954102, rows: 665, samples: 4500
loading ./weights/stage_2_gauss_multivariate_fold_1_of_5
1/1 [==============================] - 0s 19ms/step
finished 1 after 1.1946825981140137, rows: 665, samples: 4500
loading ./weights/stage_2_gauss_multivariate_fold_2_of_5
1/1 [==============================] - 0s 19ms/step
finished 2 after 1.4869976043701172, rows: 665, samples: 4500
loading ./weights/stage_2_gauss_multivariate_fold_3_of_5
1/1 [==============================] - 0s 18ms/step
finished 3 after 1.1797845363616943, rows: 665, samples: 4500
loading ./weights/stage_2_gauss_multivariate_fold_4_of_5
1/1 [==============================] - 0s 18ms/step


C:\Users\andre\AppData\Local\Temp\ipykernel_30688\102697444.py:42: RuntimeWarning: covariance is not positive-semidefinite.
  _y_pred_sample = np.random.multivariate_normal(mu[i], Sigma[i], instances)


finished 4 after 1.2572643756866455, rows: 665, samples: 4500


In [61]:
min_vals = target_scales_arr[:, 0]
max_vals = target_scales_arr[:, 1]
_y_pred_test = g_y_pred_test.reshape(-1, num_targets)
g_y_pred_test_org = (_y_pred_test*(max_vals - min_vals)) + min_vals # TODO: wrap this in a function
g_y_pred_test_org = g_y_pred_test_org.reshape(len(model_checkpoints)*instances, len(test_idx), num_targets)
g_y_pred_test_org = np.swapaxes(g_y_pred_test_org, 1, 0)

In [63]:
mg_individual_model_outputs = []
mg_folded_model_outputs = []
mg_best_model_outputs = []
mg_merge_with_uniform_model_outputs = []

indivual_model_sample_size = 2500
folded_model_sample_size = 500
final_grouping = 1000
group_with_uniform_sample_size = 500
sampled_sizes = [indivual_model_sample_size, folded_model_sample_size, final_grouping, group_with_uniform_sample_size]
cumulative_sizes = np.cumsum(sampled_sizes)
sampled_size = np.sum(sampled_sizes)
for i in range(5):
    _mg_y_preds = g_y_pred_test_org[:, i*sampled_size:(i+1)*sampled_size, :].copy()
    mg_individual_model_outputs.append(_mg_y_preds[:, :cumulative_sizes[0], :])
    mg_folded_model_outputs.append(_mg_y_preds[:, cumulative_sizes[0]:cumulative_sizes[1], :])
    mg_best_model_outputs.append(_mg_y_preds[:, cumulative_sizes[1]:cumulative_sizes[2], :])
    mg_merge_with_uniform_model_outputs.append(_mg_y_preds[:, cumulative_sizes[2]:, :])
    
mg_folded_model_outputs_arr = np.concatenate(mg_folded_model_outputs, axis=1)
mg_best_model_outputs_arr = np.concatenate([mg_best_model_outputs[0], mg_best_model_outputs[4]], axis=1)

In [ ]:
# mg_avg_posterior_scores = []
# mg_posterior_scores_list = []
# for _pred_arr in mg_individual_model_outputs + [mg_folded_model_outputs_arr, mg_best_model_outputs_arr]:
#     tr1 = _pred_arr
#     # assume equal importance weights
#     weights1 = np.ones((tr1.shape[0], tr1.shape[1]))/np.sum(np.ones(tr1.shape[1]))

#     posterior_scores = []
#     bounds_matrix = default_prior_bounds()
#     for idx, pl_idx in enumerate(test_idx):
#         tr_GT = trace_GT[f'Planet_train{pl_idx+1}']['tracedata'][()]
#         weights_GT = trace_GT[f'Planet_train{pl_idx+1}']['weights'][()]
#         # ignore rows missing ground truth
#         if np.isnan(tr_GT).sum() == 1:
#             continue
#         # compute posterior loss
#         score = compute_posterior_loss(tr1[idx], weights1[idx], tr_GT, weights_GT, bounds_matrix)
#         posterior_scores.append(score)

#     avg_posterior_score = np.mean(posterior_scores)
#     print(avg_posterior_score)
#     mg_avg_posterior_scores.append(avg_posterior_score)
#     mg_posterior_scores_list.append(posterior_scores)

In [ ]:
# sns.histplot(mg_posterior_scores_list[-1])

In [47]:
# tr1 = g_y_pred_test_org
# weights1 = np.ones((tr1.shape[0], tr1.shape[1]))/np.sum(np.ones(tr1.shape[1]))

# posterior_scores = []
# bounds_matrix = default_prior_bounds()
# for idx, pl_idx in enumerate(test_idx):
#     tr_GT = trace_GT[f'Planet_train{pl_idx+1}']['tracedata'][()]
#     weights_GT = trace_GT[f'Planet_train{pl_idx+1}']['weights'][()]
#     ## skip cases with no ground truth
#     if np.isnan(tr_GT).sum() == 1:
#         continue
#     # compute posterior loss
#     score = compute_posterior_loss(tr1[idx], weights1[idx], tr_GT, weights_GT, bounds_matrix)
#     posterior_scores.append(score)

# avg_posterior_score = np.mean(posterior_scores)

# avg_posterior_score

# Uniform check

In [64]:
scaler_pickles = [
    'uniform_percentile_fold_0_of_5.pickle',
    'uniform_percentile_fold_1_of_5.pickle',
    'uniform_percentile_fold_2_of_5.pickle',
    'uniform_percentile_fold_3_of_5.pickle',
    'uniform_percentile_fold_4_of_5.pickle'
]

model_checkpoints = [
    './weights/stage_2_uniform_quantiles_fold_0_of_5',
    './weights/stage_2_uniform_quantiles_fold_1_of_5',
    './weights/stage_2_uniform_quantiles_fold_2_of_5',
    './weights/stage_2_uniform_quantiles_fold_3_of_5',
    './weights/stage_2_uniform_quantiles_fold_4_of_5'
]

In [66]:
loss = 'uniform_quantiles'
n_feature_cols = None
num_spectra_features = wl_channels
quantile_percents = [0.05, 0.5, 0.95]

num_targets = len(target_labels)
num_rows = test_spectra.shape[0]
num_samples = 3500 # 2500 + 500 + 500 for each fold 

y_pred_scaled = np.zeros((num_rows, num_samples*len(model_checkpoints), num_targets))

model = get_model(loss, num_spectra_features, n_feature_cols, num_added_features, num_targets, 
                  target_labels=target_labels, quantile_percents=quantile_percents)
    
for j, (scaler_file, model_ckpt) in enumerate(zip(scaler_pickles, model_checkpoints)):
    start_time = time.time()
    with open('saved_objects/' + scaler_file, 'rb') as f:
        aux_data_scaler = pickle.load(f)
        
    _aux_data = get_data(aux_data, spectra)
    aux_columns = _aux_data.columns
    test_aux_data = _aux_data[aux_columns].iloc[test_idx]
    
    # num_added_features = len(aux_columns) - 1
    std_test_aux_data = aux_data_scaler.transform(test_aux_data.drop(['planet_ID'], axis=1).values)
    print(f'loading {model_ckpt}')
    model.load_weights(model_ckpt).expect_partial()
    
    _test_spectra = test_spectra - np.mean(test_spectra, axis=1).reshape((-1, 1))
    _x_data = [_test_spectra, std_test_aux_data]
    y_pred = model.predict(_x_data, verbose=1, batch_size=5000)
    for t_id in range(len(target_labels)):
        for i in range(num_samples):
            y_pred_scaled[:, i + j*num_samples, t_id] += np.random.uniform(low=y_pred[t_id][:, 0], high=y_pred[t_id][:, -1], size=None)
            
    print(f'finished {j} after {time.time() - start_time}, rows: {len(test_idx)}, samples: {num_samples}')

del(model)
tf.keras.backend.clear_session()
            
y_pred_unscaled = (y_pred_scaled*(max_vals - min_vals)) + min_vals

loading ./weights/stage_2_uniform_quantiles_fold_0_of_5
1/1 [==============================] - 1s 510ms/step
finished 0 after 1.3817200660705566, rows: 665, samples: 4500
loading ./weights/stage_2_uniform_quantiles_fold_1_of_5
1/1 [==============================] - 0s 17ms/step
finished 1 after 0.8819115161895752, rows: 665, samples: 4500
loading ./weights/stage_2_uniform_quantiles_fold_2_of_5
1/1 [==============================] - 0s 18ms/step
finished 2 after 0.8535933494567871, rows: 665, samples: 4500
loading ./weights/stage_2_uniform_quantiles_fold_3_of_5
1/1 [==============================] - 0s 17ms/step
finished 3 after 0.8980004787445068, rows: 665, samples: 4500
loading ./weights/stage_2_uniform_quantiles_fold_4_of_5
1/1 [==============================] - 0s 17ms/step
finished 4 after 0.8352808952331543, rows: 665, samples: 4500


In [76]:
individual_model_outputs = []
folded_model_outputs = []
merge_with_mg_model_outputs = []
indivual_model_sample_size = 2500
folded_model_sample_size = 500
merge_with_mg_sample_size = 500
sampled_sizes = [indivual_model_sample_size, folded_model_sample_size, merge_with_mg_sample_size]
cumulative_sizes = np.cumsum(sampled_sizes)
sampled_size = np.sum(sampled_sizes)
for i in range(5):
    _y_preds = y_pred_unscaled[:, i*sampled_size:(i+1)*sampled_size, :].copy()
    individual_model_outputs.append(_y_preds[:, :cumulative_sizes[0], :])
    folded_model_outputs.append(_y_preds[:, cumulative_sizes[0]:cumulative_sizes[1], :])
    merge_with_mg_model_outputs.append(_y_preds[:, cumulative_sizes[1]:, :])
    
folded_model_outputs_arr = np.concatenate(folded_model_outputs, axis=1)

In [54]:
# avg_posterior_scores = []
# posterior_scores_list = []
# for _pred_arr in individual_model_outputs + [folded_model_outputs_arr]:
#     tr1 = _pred_arr
#     # assume equal importance weights
#     weights1 = np.ones((tr1.shape[0], tr1.shape[1]))/np.sum(np.ones(tr1.shape[1]))

#     posterior_scores = []
#     planet_ids = []
#     bounds_matrix = default_prior_bounds()
#     for idx, pl_idx in enumerate(test_idx):
#         tr_GT = trace_GT[f'Planet_train{pl_idx+1}']['tracedata'][()]
#         weights_GT = trace_GT[f'Planet_train{pl_idx+1}']['weights'][()]
#         # ignore rows missing ground truth
#         if np.isnan(tr_GT).sum() == 1:
#             continue
#         # compute posterior loss
#         score = compute_posterior_loss(tr1[idx], weights1[idx], tr_GT, weights_GT, bounds_matrix)
#         posterior_scores.append(score)
#         planet_ids.append(pl_idx)

#     avg_posterior_score = np.mean(posterior_scores)
#     print(avg_posterior_score)
#     avg_posterior_scores.append(avg_posterior_score)
#     posterior_scores_list.append(posterior_scores)

662.6361331901182
660.1087862513427
662.8707196562837
660.0282706766918
653.3791621911922
696.4848979591836


In [47]:
data_dict = {
    f'model_{i}': scores for i, scores in enumerate(posterior_scores_list)
}
df_uniform_posterior_scores = pd.DataFrame(data_dict)
df_uniform_posterior_scores['planet_idx'] = planet_ids
df_uniform_posterior_scores['planet'] = df_uniform_posterior_scores['planet_idx'].apply(lambda x: f'Planet_train{x+1}')

df_uniform_posterior_scores.head()

,model_0,model_1,model_2,model_3,model_4,model_5,planet_idx,planet
0,760.285714,732.800000,718.114286,822.857143,716.000000,793.714286,4449,Planet_train4450
1,470.342857,467.371429,428.285714,413.885714,506.285714,528.571429,4178,Planet_train4179
2,652.628571,641.257143,624.800000,608.971429,571.028571,679.828571,40300,Planet_train40301
3,710.457143,649.828571,602.171429,710.171429,656.800000,713.942857,3161,Planet_train3162
4,791.600000,758.057143,833.200000,756.000000,778.057143,810.571429,3601,Planet_train3602


In [48]:
df_uniform_posterior_scores.to_csv('uniform_models_posterior_scores.csv', index=False)

In [80]:
# sns.histplot(posterior_scores_list[-1])

In [30]:
# tr1 = y_pred_unscaled
# # weight takes into account the importance of each point in the tracedata. 
# # for now we just assume them to be equally weighted
# weights1 = np.ones((tr1.shape[0], tr1.shape[1]))/np.sum(np.ones(tr1.shape[1]))

# posterior_scores = []
# bounds_matrix = default_prior_bounds()
# for idx, pl_idx in enumerate(val_idx):
#     tr_GT = trace_GT[f'Planet_train{pl_idx+1}']['tracedata'][()]
#     weights_GT = trace_GT[f'Planet_train{pl_idx+1}']['weights'][()]
#     ## there are cases without ground truth, we will skip over them for this baseline
#     ## but every example in leaderboard and final evaluation set will have a complementary ground truth
#     if np.isnan(tr_GT).sum() == 1:
#         continue
#     # compute posterior loss
#     score = compute_posterior_loss(tr1[idx], weights1[idx], tr_GT, weights_GT, bounds_matrix)
#     posterior_scores.append(score)

# avg_posterior_score = np.mean(posterior_scores)
# print(avg_posterior_score)
# sns.histplot(posterior_scores)

# Full evaluation

In [77]:
mg_outputs_dict = {f'mg_fold_{k}': _arr for k, _arr in enumerate(mg_individual_model_outputs)}
mg_outputs_dict['mg_all_folds'] = mg_folded_model_outputs_arr
mg_outputs_dict['mg_best_folds'] = mg_best_model_outputs_arr

uniform_outputs_dict = {f'uniform_fold_{k}': _arr for k, _arr in enumerate(individual_model_outputs)}
uniform_outputs_dict['uniform_all_folds'] = folded_model_outputs_arr

total_outputs_dict = {**mg_outputs_dict, **uniform_outputs_dict}
_arr = merge_with_mg_model_outputs + [mg_merge_with_uniform_model_outputs[0], mg_merge_with_uniform_model_outputs[4]]
total_outputs_dict['best_mg_folds_and_all_uniform_folds'] = np.concatenate(_arr, axis=1)
total_outputs_dict.keys()

dict_keys(['mg_fold_0', 'mg_fold_1', 'mg_fold_2', 'mg_fold_3', 'mg_fold_4', 'mg_all_folds', 'mg_best_folds', 'uniform_fold_0', 'uniform_fold_1', 'uniform_fold_2', 'uniform_fold_3', 'uniform_fold_4', 'uniform_all_folds', 'best_mg_folds_and_all_uniform_folds'])

In [79]:
avg_posterior_scores = []
posterior_scores_list = []
for _model, _pred_arr in total_outputs_dict.items():
    print(f'processing {_model}')
    tr1 = _pred_arr
    print(f'\ttrace shape: {tr1.shape}')
    
    # assume equal importance weights
    weights1 = np.ones((tr1.shape[0], tr1.shape[1]))/np.sum(np.ones(tr1.shape[1]))

    posterior_scores = []
    planet_ids = []
    bounds_matrix = default_prior_bounds()
    for idx, pl_idx in enumerate(test_idx):
        tr_GT = trace_GT[f'Planet_train{pl_idx+1}']['tracedata'][()]
        weights_GT = trace_GT[f'Planet_train{pl_idx+1}']['weights'][()]
        
        # ignore rows missing ground truth
        if np.isnan(tr_GT).sum() == 1:
            continue
            
        # compute posterior loss
        score = compute_posterior_loss(tr1[idx], weights1[idx], tr_GT, weights_GT, bounds_matrix)
        posterior_scores.append(score)
        planet_ids.append(pl_idx)

    avg_posterior_score = np.mean(posterior_scores)
    print(f'\tavg_posterior_score: {avg_posterior_score}')
    avg_posterior_scores.append(avg_posterior_score)
    posterior_scores_list.append(posterior_scores)

processing mg_fold_0
	trace shape: (665, 2500, 7)
	avg_posterior_score: 686.2922448979592
processing mg_fold_1
	trace shape: (665, 2500, 7)
	avg_posterior_score: 568.0599785177229
processing mg_fold_2
	trace shape: (665, 2500, 7)
	avg_posterior_score: 488.0128893662728
processing mg_fold_3
	trace shape: (665, 2500, 7)
	avg_posterior_score: 483.5508485499463
processing mg_fold_4
	trace shape: (665, 2500, 7)
	avg_posterior_score: 632.1612889366272
processing mg_all_folds
	trace shape: (665, 2500, 7)
	avg_posterior_score: 615.0172287862513
processing mg_best_folds
	trace shape: (665, 2000, 7)
	avg_posterior_score: 690.7070891514501
processing uniform_fold_0
	trace shape: (665, 2500, 7)
	avg_posterior_score: 662.7546294307197
processing uniform_fold_1
	trace shape: (665, 2500, 7)
	avg_posterior_score: 660.1757250268529
processing uniform_fold_2
	trace shape: (665, 2500, 7)
	avg_posterior_score: 662.9932545649839
processing uniform_fold_3
	trace shape: (665, 2500, 7)
	avg_posterior_score: 6

In [106]:
data_dict = {
    m: scores for m, scores in zip(total_outputs_dict.keys(), posterior_scores_list)
}
df_posterior_scores = pd.DataFrame(data_dict)
df_posterior_scores['planet_idx'] = planet_ids
df_posterior_scores['planet'] = df_posterior_scores['planet_idx'].apply(lambda x: f'Planet_train{x+1}')

df_posterior_scores.head()

,mg_fold_0,mg_fold_1,mg_fold_2,mg_fold_3,mg_fold_4,mg_all_folds,mg_best_folds,uniform_fold_0,uniform_fold_1,uniform_fold_2,uniform_fold_3,uniform_fold_4,uniform_all_folds,best_mg_folds_and_all_uniform_folds,planet_idx,planet
0,729.885714,430.514286,535.314286,556.171429,712.742857,651.485714,742.428571,759.600000,721.371429,726.742857,820.457143,715.371429,797.028571,778.204082,4449,Planet_train4450
1,454.457143,529.771429,465.885714,421.314286,628.114286,560.571429,554.285714,475.942857,461.657143,426.971429,416.000000,514.800000,525.314286,534.693878,4178,Planet_train4179
2,650.171429,498.800000,416.342857,413.828571,484.514286,614.571429,635.071429,648.000000,638.800000,622.571429,608.342857,564.228571,687.771429,663.632653,40300,Planet_train40301
3,791.257143,678.114286,541.600000,519.828571,761.257143,691.485714,831.214286,711.257143,651.600000,599.257143,708.228571,658.857143,712.800000,719.510204,3161,Planet_train3162
4,703.657143,682.057143,485.028571,402.857143,727.257143,633.314286,718.500000,790.857143,754.742857,834.000000,757.600000,781.600000,801.371429,796.897959,3601,Planet_train3602


In [107]:
df_posterior_scores.to_csv('histories/posterior_scores.csv', index=False)

### Spectral scores

In [87]:
N_examples = 10 ## number of test examples to go through
N_samples = 10 ## number of quantiles to sample (fixed to 10 in the competition)
q_list = np.linspace(0.01, 0.99, N_samples)
## beta - weight of the posterior loss [0,1], and the weight of spectral loss will decrease accordingly. 
beta = 0.8

RJUP = 69911000
MJUP = 1.898e27
RSOL = 696340000

In [88]:
## Path variables
opacity_path = "../ADC_t3_input/xsec/"
CIA_path = "../ADC_t3_input/cia/HITRAN"

In [92]:
## read in spectral grid
ariel_wlgrid, ariel_wlwidth, ariel_wngrid, ariel_wnwidth = ariel_resolution()
## Initialise base T3 model for ADC2023
fm = initialise_forward_model(opacity_path, CIA_path)

In [93]:
# raed auxillary information from the input file (Provided from ADC2023)
aux_df = aux_data.iloc[test_idx]
# ensure the dimensionality matches forward model's input.
Rs = aux_df['star_radius_m']/RSOL
# Rp = aux_df['planet_radius_m']/RJUP
Mp = aux_df['planet_mass_kg']/MJUP

In [100]:
spectral_scores_dict = {}
break_idx = 20
for _model, tr1 in total_outputs_dict.items():
    
    ## read in spectral grid
    ariel_wlgrid, ariel_wlwidth, ariel_wngrid, ariel_wnwidth = ariel_resolution()
    ## Initialise base T3 model for ADC2023
    fm = initialise_forward_model(opacity_path, CIA_path)

    print(f'processing {_model}')
    start_time = time.time()
    weights1 = np.ones((tr1.shape[0], tr1.shape[1]))/np.sum(np.ones(tr1.shape[1]))

    spectral_scores = []
    bounds_matrix = default_prior_bounds()
    for idx, pl_idx in enumerate(test_idx):
        ## put an early stop here as it will take forever to go through 5000 examples. 
        if idx == break_idx:
            break

        tr_GT = trace_GT[f'Planet_train{pl_idx+1}']['tracedata'][()]
        weights_GT = trace_GT[f'Planet_train{pl_idx+1}']['weights'][()]

        # again to avoid unlabelled data
        if np.isnan(tr_GT).sum() == 1:
            continue

        proxy_compute_spectrum = setup_dedicated_fm(fm, pl_idx, Rs, Mp, ariel_wngrid, ariel_wnwidth)

        score = compute_spectral_loss(tr1[idx], weights1[idx], tr_GT, weights_GT, bounds_matrix, proxy_compute_spectrum, q_list)
        spectral_scores.append(score)
        
    spectral_scores_dict[_model] = spectral_scores
    avg_spectral_score = np.mean(spectral_scores)
    print(f'\tfinished after {time.time() - start_time}, score: {avg_spectral_score}')

processing mg_fold_0
	finished after 1178.2593631744385, score: 957.5226928839587
processing mg_fold_1


C:\Users\andre\anaconda3\envs\tf_bleeding\lib\site-packages\taurex\model\simplemodel.py:309: RuntimeWarning: divide by zero encountered in true_divide
  return (self.pressureProfile)/(KBOLTZ*self.temperatureProfile)


	finished after 1177.76735830307, score: 910.9787443422043
processing mg_fold_2
	finished after 1175.281938791275, score: 904.5191545363514
processing mg_fold_3
	finished after 1179.2760655879974, score: 878.525482483439
processing mg_fold_4
	finished after 1174.5453746318817, score: 947.8090968601093
processing mg_all_folds
	finished after 1177.6975753307343, score: 919.5214968099411
processing mg_best_folds
	finished after 1177.2504661083221, score: 944.3199240293783
processing uniform_fold_0
	finished after 1173.3180530071259, score: 866.9255286109867
processing uniform_fold_1
	finished after 1173.0261752605438, score: 896.2629955846307
processing uniform_fold_2
	finished after 1178.754096031189, score: 899.0930476287149
processing uniform_fold_3
	finished after 1180.5499951839447, score: 897.607662401769
processing uniform_fold_4
	finished after 1177.5019390583038, score: 895.2954473430315
processing uniform_all_folds
	finished after 1174.2739367485046, score: 880.1497518877807
pro

In [111]:
data_dict = {
    m: scores for m, scores in spectral_scores_dict.items()
}
df_spectral_scores = pd.DataFrame(data_dict)
df_spectral_scores['planet_idx'] = planet_ids[:break_idx]
df_spectral_scores['planet'] = df_spectral_scores['planet_idx'].apply(lambda x: f'Planet_train{x+1}')

df_spectral_scores.head()

,mg_fold_0,mg_fold_1,mg_fold_2,mg_fold_3,mg_fold_4,mg_all_folds,mg_best_folds,uniform_fold_0,uniform_fold_1,uniform_fold_2,uniform_fold_3,uniform_fold_4,uniform_all_folds,best_mg_folds_and_all_uniform_folds,planet_idx,planet
0,952.201680,949.339451,881.959938,867.636232,960.755411,921.445083,918.318939,849.863256,874.229691,893.023334,828.178355,886.750861,884.820104,810.976895,4449,Planet_train4450
1,950.029558,935.977619,858.241188,835.340677,908.222888,895.379315,949.248473,910.916970,912.018312,916.742764,848.512629,861.548007,876.055451,883.248706,4178,Planet_train4179
2,958.507064,955.124329,898.463089,900.969192,952.263389,899.657768,950.478039,863.731255,886.121280,852.081369,849.691788,958.046511,850.318887,824.582117,40300,Planet_train40301
3,959.052647,905.483897,891.432166,900.978088,949.049141,890.762040,952.955071,876.598214,909.930267,925.109010,910.716040,875.660351,864.294612,857.223097,3161,Planet_train3162
4,983.491578,982.571990,964.037836,969.319307,979.004825,966.370451,973.763028,975.537935,963.855397,953.846054,953.029152,959.018311,947.580263,958.546885,3601,Planet_train3602


In [112]:
df_spectral_scores.to_csv('histories/spectral_scores.csv', index=False)